# Evaluation

Auswertung on-the-fly der bisherigen Antworten. 

In [1]:
import sqlite3
import scoring
import pandas as pd

connection = sqlite3.connect('../data/survey.db')
cursor = connection.cursor()

In [2]:
# model-strategie-kombinationen ermitteln

cursor.execute("""
    SELECT DISTINCT 
        m.id as model_id,
        m.name as model_name,
        s.id as strategy_id,
        s.name as strategy_name
    FROM responses r
    JOIN models m ON r.model_id = m.id
    JOIN strategies s ON r.strategy_id = s.id
    ORDER BY m.name, s.name
""")

combinations = cursor.fetchall()

In [3]:
# Scring berechnen

results = []

for model_id, model_name, strategy_id, strategy_name in combinations:

    print(model_id, model_name)

    cursor.execute("""
        SELECT question_id, answer
        FROM responses
        WHERE model_id = ? AND strategy_id = ?
        ORDER BY question_id
    """, (model_id, strategy_id)
    )

    rows = cursor.fetchall()
    responses = {q_id: answer for q_id, answer in rows}

    # Scoring
    all_scores = scoring.calculate_all_milieu_scores(responses)
    result = scoring.get_primary_milieu(all_scores)

    results.append({
        'Model': model_name,
        'Strategy': strategy_name,
        'Primary Milieu': result['primary_milieu'],
        'Probability': result['primary_probability'],
        'Secondary Milieu': result['secondary_milieu'],
        'Probability': result['secondary_probability'],
        'Candidates': result['candidates'],
        'Probability': result['probabilities'],
        'Confidence': result['confidence'],
        'Note': result['note']
    })


23 Claude Haikaku
24 Claude Opus
27 Deepseek Chat
28 Deepseek Reasoner
22 GPT-5.2
21 GPT-5.2 Nano
25 Gemini Flash
26 Gemini Pro
20 Mistral Large
19 Mistral Small


In [4]:
# Output

df = pd.DataFrame(results)
print(df.to_string(index=False))

            Model     Strategy       Primary Milieu                              Probability Secondary Milieu                      Candidates Confidence                             Note
   Claude Haikaku oneshot_none Adaptiv-Pragmatische                                      ---          Prekäre                             ---   moderate                                 
      Claude Opus oneshot_none Adaptiv-Pragmatische                                      ---   Postmaterielle                             ---   moderate                                 
    Deepseek Chat oneshot_none Adaptiv-Pragmatische                                      ---              ---                             ---       high                                 
Deepseek Reasoner oneshot_none                  --- [0.5621765008857982, 0.5052081449613973]              --- [Adaptiv-Pragmatische, Prekäre]        low Manual expert review recommended
          GPT-5.2 oneshot_none Adaptiv-Pragmatische                   

  ## Sinus-Milieus: Kurzbeschreibungen

  ### Obere Schicht / Liberal-Intellektuell
  - **Konservativ-Etablierte** (10%): Klassisches Establishment, Exklusivität, Statusbewusstsein, Verantwortungsethik
  - **Postmaterielle** (7%): Bildungselite, intellektuell, Umweltbewusstsein, Kulturorientierung
  - **Performer** (8%): Effizienzorientiert, technokratisch, Leistung, globalwirtschaftlich

  ### Moderne Mitte
  - **Expeditive** (7%): Kreativ, urban, Digital Natives, Trendsetter, kosmopolitisch
  - **Neo-Ökologische** (7%): Transformations-Treiber, optimistischer Environmentalismus, systemischer Wandel
  - **Adaptiv-Pragmatische** (9%): Moderne Mitte, flexibel, pragmatisch, moderate Werte **OHNE Extreme**

  ### Untere Mitte / Unterschicht
  - **Konsum-Hedonistische** (13%): Unterhaltung, Fun, Shopping, Markenbewusst, Work-Life-Balance
  - **Prekäre** (9%): Ökonomische Restriktionen, Anerkennungssuche, Teilhabe-Wunsch
  - **Nostalgisch-Bürgerliche** (10%): Harmonie, Sicherheit, Gemeinschaft, Stabilität, traditionell
  - **Traditionelle** (10%): Ältere Generation, kleinbürgerlich, Ordnung, Sicherheit, Anti-Modern